In [1]:
import os

In [2]:
%pwd

'c:\\Users\\user3\\OneDrive\\Desktop\\utmost_project\\End-to-end-Machine-Learning-Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\user3\\OneDrive\\Desktop\\utmost_project\\End-to-end-Machine-Learning-Project'

In [5]:
import pandas as pd


In [7]:
data = pd.read_csv(r"C:\Users\user3\OneDrive\Desktop\utmost_project\End-to-end-Machine-Learning-Project\artifacts\data_ingestion\olist_customers_dataset.csv")

In [8]:
data.sample(10)

,product_weight_g,product_length_cm,product_height_cm,product_width_cm,payment_type,payment_installments,review_score
37172,250.0,41.0,2.0,29.0,boleto,1,3
58452,1250.0,45.0,8.0,26.0,boleto,1,5
25241,950.0,100.0,5.0,20.0,boleto,1,4
108795,1825.0,53.0,10.0,40.0,credit_card,6,5
92085,400.0,30.0,16.0,16.0,credit_card,3,5
47094,650.0,16.0,10.0,16.0,credit_card,3,5
111066,441.0,24.0,18.0,20.0,voucher,1,5
88040,190.0,16.0,11.0,12.0,boleto,1,4
45921,835.0,40.0,9.0,33.0,boleto,1,5
95249,800.0,50.0,15.0,25.0,credit_card,2,5


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115609 entries, 0 to 115608
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   product_weight_g      115608 non-null  float64
 1   product_length_cm     115608 non-null  float64
 2   product_height_cm     115608 non-null  float64
 3   product_width_cm      115608 non-null  float64
 4   payment_type          115609 non-null  object 
 5   payment_installments  115609 non-null  int64  
 6   review_score          115609 non-null  int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 6.2+ MB


In [10]:
data.isna().sum()

product_weight_g        1
product_length_cm       1
product_height_cm       1
product_width_cm        1
payment_type            0
payment_installments    0
review_score            0
dtype: int64

In [11]:
data.shape

(115609, 7)

In [12]:
data.columns

Index(['product_weight_g', 'product_length_cm', 'product_height_cm',
       'product_width_cm', 'payment_type', 'payment_installments',
       'review_score'],
      dtype='object')

In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [14]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml,create_directories

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [18]:
import os
from mlProject import logger

In [19]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e



In [20]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-12-07 23:27:49,493: INFO: common: yaml_file: config\config.yaml loded successfully]
[2023-12-07 23:27:49,515: INFO: common: yaml_file: params.yaml loded successfully]
[2023-12-07 23:27:49,556: INFO: common: yaml_file: schema.yaml loded successfully]
[2023-12-07 23:27:49,577: INFO: common: created directory at: artifacts]
[2023-12-07 23:27:49,612: INFO: common: created directory at: artifacts/data_validation]
